In [ ]:
# from https://github.com/H-arshit/UNET-On-COCO/blob/master/tf_Keras_COCO_UNET.ipynb
# Importing Data From COCO

from pycocotools import coco, cocoeval, _mask
from pycocotools import mask as maskUtils 
import array
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import os
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
%matplotlib inline

In [ ]:
COCO_DIR = 'coco-2017'
TRAIN_IMG = COCO_DIR + '/train/data'
VAL_IMG = COCO_DIR + '/validation/data'

In [ ]:
CATEGORY_NAMES=['person']

ANNOTATION_FILE_VAL = 'coco-2017/raw/instances_val2017.json'
ANNOTATION_FILE_TRAIN = 'coco-2017/raw/instances_train2017.json'


coco_train = coco.COCO(ANNOTATION_FILE_TRAIN)
catIds_train = coco_train.getCatIds(catNms=CATEGORY_NAMES);
imgIds_train = coco_train.getImgIds(catIds=catIds_train);
imgDict_train = coco_train.loadImgs(imgIds_train)
len(imgIds_train) , len(catIds_train)


coco_val = coco.COCO(ANNOTATION_FILE_VAL)
catIds_val = coco_val.getCatIds(catNms=CATEGORY_NAMES);
imgIds_val = coco_val.getImgIds(catIds=catIds_val);
imgDict_val = coco_val.loadImgs(imgIds_val)
len(imgIds_val) , len(catIds_val)

In [ ]:
from random import shuffle

shuffle(imgIds_train)
shuffle(imgIds_val)

imgIds_train = imgIds_train[0:6000]
imgIds_val = imgIds_val[0:600]

In [ ]:
train_images_person = ["{0:012d}.jpg".format(ids) for ids in imgIds_train]
del_img_train = set(os.listdir("coco-2017/train/data")) - set(train_images_person)
for file_name in del_img_train:
    file_name = "coco-2017/train/data/" + file_name
    if os.path.exists(file_name):
        os.remove(file_name)

In [ ]:
print(len(os.listdir("coco-2017/train/data")))

In [ ]:
val_images_person = ["{0:012d}.jpg".format(ids) for ids in imgIds_val]
del_img_val = set(os.listdir(VAL_IMG)) - set(val_images_person)
i = 1
for file_name in del_img_val:
    file_name = VAL_IMG + "/" + file_name
    if os.path.exists(file_name):
        print(i)
        os.remove(file_name)
        i+=1

len(os.listdir(VAL_IMG))

In [ ]:
from PIL import Image

In [ ]:
!mkdir mask_train_2017

In [ ]:
count = 0 

for ID in imgIds_train:

    file_path = "./mask_train_2017/COCO_train2017_{0:012d}.jpg".format(ID)
  
    sampleImgIds = coco_train.getImgIds(imgIds = [ID])
    sampleImgDict = coco_train.loadImgs(sampleImgIds[np.random.randint(0,len(sampleImgIds))])[0]

    annIds = coco_train.getAnnIds(imgIds=sampleImgDict['id'], catIds=catIds_train, iscrowd=0)
    anns = coco_train.loadAnns(annIds)


    mask = coco_train.annToMask(anns[0])
    for i in range(len(anns)):
        mask = mask | coco_train.annToMask(anns[i])
  
    mask = Image.fromarray(mask * 255 , mode = "L")
    mask.save(file_path)
    count = count + 1
    print(count)

In [ ]:
!mkdir mask_val_2017

In [ ]:
count = 0 
for ID in imgIds_val:

    file_path = "./mask_val_2017/COCO_val2017_{0:012d}.jpg".format(ID)
  
    sampleImgIds = coco_val.getImgIds(imgIds = [ID])
    sampleImgDict = coco_val.loadImgs(sampleImgIds[np.random.randint(0,len(sampleImgIds))])[0]

    annIds = coco_val.getAnnIds(imgIds=sampleImgDict['id'], catIds=catIds_val, iscrowd=0)
    anns = coco_val.loadAnns(annIds)


    mask = coco_val.annToMask(anns[0])
    for i in range(len(anns)):
        mask = mask | coco_val.annToMask(anns[i])
  
    mask = Image.fromarray(mask * 255 , mode = "L")
    mask.save(file_path)
  
    count = count + 1
    print(count)

# Data Generator

In [ ]:
import os
import sys
import random

import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *


seed = 0

random.seed = seed
np.random.seed = seed 

In [ ]:
class DataGen(tf.keras.utils.Sequence):
  
    def __init__(self , path_input , path_mask , batch_size = 8 , image_size = 128 , split='train'):
    
        self.ids = os.listdir(path_input)
        self.path_input = path_input
        self.path_mask = path_mask
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
        self.split = split
  
    def __load__(self , id_name):
    
        image_path = os.path.join(self.path_input , id_name)
        mask_path = os.path.join(self.path_mask , "COCO_{}2017_{}".format(self.split, id_name)) 
    
        image = cv2.imread(image_path , 1) # 1 specifies RGB format
        image = cv2.resize(image , (self.image_size , self.image_size)) # resizing before inserting to the network
    
        mask = cv2.imread(mask_path)
        mask = cv2.resize(mask , (self.image_size , self.image_size))
        mask = mask[:, :, 0].reshape((self.image_size , self.image_size , 1))
      
        #normalize image
        image = image / 255.0
        mask = mask / 255.0
    
        return image , mask
  
    def __getitem__(self , index):
    
        if (index + 1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index * self.batch_size
        
        file_batch = self.ids[index * self.batch_size : (index + 1) * self.batch_size]
    
        images = []
        masks = []
    
        for id_name in file_batch : 
      
            _img , _mask = self.__load__(id_name)
            print(_img.shape)
            images.append(_img)
            masks.append(_mask)
    
    
        images = np.array(images)
        masks = np.array(masks)
    
    
        return images , masks
  
  
    def on_epoch_end(self):
        pass
  
  
    def __len__(self):
    
        return int(np.ceil(len(self.ids) / float(self.batch_size)))

# UNet

In [ ]:
def down_block(
    input_tensor,
    no_filters,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    kernel_initializer="he_normal",
    max_pool_window=(2, 2),
    max_pool_stride=(2, 2)
):
    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(input_tensor)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    # conv for skip connection
    conv = Activation("relu")(conv)

    pool = MaxPooling2D(pool_size=max_pool_window, strides=max_pool_stride)(conv)

    return conv, pool

In [ ]:
def bottle_neck(
    input_tensor,
    no_filters,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    kernel_initializer="he_normal"
):
    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(input_tensor)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    return conv

In [ ]:
def up_block(    
    input_tensor,
    no_filters,
    skip_connection, 
    kernel_size=(3, 3),
    strides=(1, 1),
    upsampling_factor = (2,2),
    max_pool_window = (2,2),
    padding="same",
    kernel_initializer="he_normal"):
    
    
    conv = Conv2D(
        filters = no_filters,
        kernel_size= max_pool_window,
        strides = strides,
        activation = None,
        padding = padding,
        kernel_initializer=kernel_initializer
    )(UpSampling2D(size = upsampling_factor)(input_tensor))
    
    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv) 
    
    
    conv = concatenate( [skip_connection , conv]  , axis = -1)
    
    
    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)

    conv = Conv2D(
        filters=no_filters,
        kernel_size=kernel_size,
        strides=strides,
        activation=None,
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)

    conv = BatchNormalization(scale=True)(conv)

    conv = Activation("relu")(conv)
    
    return conv

In [ ]:
def output_block(input_tensor,
    padding="same",
    kernel_initializer="he_normal"
):
    
    conv = Conv2D(
        filters=2,
        kernel_size=(3,3),
        strides=(1,1),
        activation="relu",
        padding=padding,
        kernel_initializer=kernel_initializer
    )(input_tensor)
    
    
    conv = Conv2D(
        filters=1,
        kernel_size=(1,1),
        strides=(1,1),
        activation="sigmoid",
        padding=padding,
        kernel_initializer=kernel_initializer
    )(conv)
    
    
    return conv

In [ ]:
def UNet(input_shape = (128,128,3)):
    
    filter_size = [64,128,256,512,1024]
    
    inputs = Input(shape = input_shape)
    
    d1 , p1 = down_block(input_tensor= inputs,
                         no_filters=filter_size[0],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    d2 , p2 = down_block(input_tensor= p1,
                         no_filters=filter_size[1],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    
    d3 , p3 = down_block(input_tensor= p2,
                         no_filters=filter_size[2],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    
    d4 , p4 = down_block(input_tensor= p3,
                         no_filters=filter_size[3],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal",
                         max_pool_window=(2,2),
                         max_pool_stride=(2,2))
    
    
    b = bottle_neck(input_tensor= p4,
                         no_filters=filter_size[4],
                         kernel_size = (3,3),
                         strides=(1,1),
                         padding="same",
                         kernel_initializer="he_normal")
    
    
    
    u4 = up_block(input_tensor = b,
                  no_filters = filter_size[3],
                  skip_connection = d4,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    u3 = up_block(input_tensor = u4,
                  no_filters = filter_size[2],
                  skip_connection = d3,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    
    u2 = up_block(input_tensor = u3,
                  no_filters = filter_size[1],
                  skip_connection = d2,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    
    u1 = up_block(input_tensor = u2,
                  no_filters = filter_size[0],
                  skip_connection = d1,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  upsampling_factor = (2,2),
                  max_pool_window = (2,2),
                  padding="same",
                  kernel_initializer="he_normal")
    
    
    
    output = output_block(input_tensor=u1 , 
                         padding = "same",
                         kernel_initializer= "he_normal")
    
    model = Model(inputs = inputs , outputs = output)
    
    
    return model

In [ ]:
model = UNet(input_shape = (128,128,3))
model.compile(optimizer = Adam(learning_rate = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
image_size = 128 
epochs = 10
batch_size = 32

In [ ]:
train_gen = DataGen(path_input = TRAIN_IMG , path_mask = './mask_train_2017' , batch_size = batch_size , image_size = image_size)
val_gen = DataGen(path_input =  VAL_IMG, path_mask =  './mask_val_2017', batch_size = batch_size , image_size = image_size, split='val')


train_steps =  len(os.listdir(TRAIN_IMG))/batch_size


model.fit(train_gen , validation_data = val_gen , steps_per_epoch = train_steps , epochs=epochs)